# Sugartrail 

In [1]:
from sugartrail import mapview, api, base
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import requests

In [2]:
%%capture
network = base.Network()

1. Insert your [Companies House API](https://developer.company-information.service.gov.uk/how-to-create-an-application) key:

In [17]:
API_input = widgets.Text(
    value='',
    placeholder='Insert API Key',
    disabled=False
)

company_text = widgets.Text(
    value='',
    placeholder='Insert Company ID',
    disabled=True
)

auth_status = widgets.HTML(
    value="",
)



auth_button = widgets.Button(description='Authenticate')
auth_button.on_click(lambda bt: auth())

init_button = widgets.Button(description='Initialise', disabled=True)
# init_button.style.button_color = '#696969'
init_button.on_click(lambda bt: init_network()) 

def auth():
    auth_button.disabled=True
    API_input.disabled=True
    api.basic_auth.username = API_input.value
    if api.test():
        auth_status.value = u'\u2705: Login successful'
        auth_button.button_style='success'
        company_text.disabled = False
        init_button.disabled = False
    else:
        auth_button.disabled=False
        API_input.disabled=False
        auth_status.value = u'\u274c: Invalid API key'

display(API_input, auth_button, auth_status)

Text(value='', placeholder='Insert API Key')

Button(description='Authenticate', style=ButtonStyle(button_color='#F5F5F5'))

HTML(value='')

2. Insert the unique company registration number (CRN) for a company you would like to investigate:

<!-- Insert a valid [Companies House Public Data API key](https://developer.company-information.service.gov.uk/get-started/) as `username` string value below. If you don't want to use the API and would prefer loading a pre-built network, uncomment and run the cell below and then run the final cell to build and load the map.  -->

In [4]:
init_status = widgets.HTML(
    value="",
) 

depth_selector = widgets.BoundedIntText(
    value=1,
    min=1,
    max=20,
    step=1,
    disabled=True
)

generate_network_button = widgets.Button(description='Build Network', disabled=True)
generate_network_button.on_click(lambda bt: generate_network()) 

def init_network():
    init_button.disabled=True
    response = api.get_company(str(company_text.value))
    company_text.disabled=True
    if response:
        network.company_id = str(company_text.value)
        init_status.value = u'\u2705: Initialisation successful for ' + str(response['company_name']) 
        init_button.button_style='success'
        navigation_button.disabled = False
        depth_selector.disabled = False
        generate_network_button.disabled = False
        
    else:
        init_button.disabled=False
        company_text.disabled=False
        init_status.value = u'\u274c: Initialisation Failed. No records for company: ' + str(company_text.value)

display(company_text, init_button, init_status)

Text(value='', disabled=True, placeholder='Insert Company ID')

Button(button_style='success', description='Initialise', disabled=True, style=ButtonStyle())

HTML(value='')

3. Select the criteria you would like to build your network from (optional):

In [6]:
navigation_button = widgets.Button(description='Update', disabled=True)
navigation_button.on_click(lambda bt: config_network()) 

officers_for_company = widgets.Checkbox(
    value=True,
    description='Get officers for each company',
    disabled=False,
    indent=False
)

historic_addresses = widgets.Checkbox(
    value=True,
    description='Get historic addresses for each company',
    disabled=False,
    indent=False
)

psc_address = widgets.Checkbox(
    value=True,
    description='Get PSC addresses for each company',
    disabled=False,
    indent=False
)

officer_appointments = widgets.Checkbox(
    value=True,
    description='Get appointments for each officer',
    disabled=False,
    indent=False
)

max_appointments_inttext = widgets.IntText(
    value=50,
    disabled=False
)

max_appointments = widgets.HBox([widgets.Label('Maximum number of appointments for each officer: '), max_appointments_inttext])

officer_correspondance = widgets.Checkbox(
    value=True,
    description='Get correspondance address for each officer',
    disabled=False,
    indent=False
)

duplicate_officers = widgets.Checkbox(
    value=True,
    description='Get duplicates for each officer',
    disabled=False,
    indent=False
)

max_duplicates_inttext = widgets.IntText(
    value=50,
    disabled=False
)

max_duplicates = widgets.HBox([widgets.Label('Maximum number of duplicate officers for each officer: '), max_duplicates_inttext])

officers_for_address = widgets.Checkbox(
    value=True,
    description='Get officers at each address',
    disabled=False,
    indent=False
)

max_officers_inttext = widgets.IntText(
    value=50,
    disabled=False
)

max_officers_at_address = widgets.HBox([widgets.Label('Maximum number of officers corresponding from address: '), max_officers_inttext])

companies_for_address = widgets.Checkbox(
    value=True,
    description='Get companies registered at each address',
    disabled=False,
    indent=False
)

max_companies_inttext = widgets.IntText(
    value=50,
    disabled=False
)

max_companies_at_address = widgets.HBox([widgets.Label('Maximum number of companies registered to address: '), max_companies_inttext])

checkboxes = [officers_for_company, 
              historic_addresses, 
              psc_address, 
              officer_appointments, 
              max_appointments, 
              officer_correspondance,
              duplicate_officers,
              max_duplicates, 
              officers_for_address,
              max_officers_at_address,
              companies_for_address,
              max_companies_at_address,
              navigation_button
             ]


In [7]:
def config_network():
    network.hop.get_company_officers = officers_for_company.value
    network.hop.get_company_address_history = historic_addresses.value
    network.hop.get_psc_correspondance_address = psc_address.value
    network.hop.get_officer_appointments = officer_appointments.value
    network.hop.officer_appointments_maxsize = max_appointments_inttext.value
    network.hop.get_officer_correspondance_address = officer_correspondance.value
    network.hop.get_officer_duplicates = duplicate_officers.value
    network.hop.officer_duplicates_maxsize = max_duplicates_inttext.value
    network.hop.get_officers_at_address = officers_for_address.value
    network.hop.officers_at_address_maxsize = max_officers_inttext.value
    network.hop.get_companies_at_address = companies_for_address.value
    network.hop.companies_at_address_maxsize = max_companies_inttext.value
    depth_selector.disabled = False
    generate_network_button.disabled = False
    navigation_button.disabled = True
    navigation_button.button_style='success'

In [8]:
accordion_config = widgets.Accordion(children=[widgets.VBox(checkboxes)])
accordion_config.set_title(0, 'Config')

In [9]:
accordion_config

Accordion(children=(VBox(children=(Checkbox(value=True, description='Get officers for each company', indent=Fa…

4. Select the depth of the network you would like to build:

In [ ]:
build_status = widgets.HTML(
    value="",
)

build_map_button=widgets.Button(description='Build Map', disabled=True)
build_map_button.on_click(lambda bt: generate_map()) 

def generate_network():
    with output_box:
        depth_selector.disabled = True
        generate_network_button.disabled = True
        network.perform_hop(depth_selector.value)
        network.run_map_preprocessing()
        build_map_button.disabled = False
        generate_network_button.button_style = "success"
        
    
output_box = widgets.Output()
display(depth_selector, generate_network_button, build_status, output_box)

5. Visualise network on a map:

In [ ]:
map_container = widgets.HTML(
    value="",
)

map_data,path_table = mapview.build_map(network, clear_widget=False) 
hbox_map = widgets.HBox([path_table])
vbox_map = widgets.VBox([map_data, hbox_map])

accordion_map = widgets.Accordion(children=[vbox_map])
accordion_map.set_title(0, 'Map')

tabs = ['Companies', 'Addresses', 'Officers', 'Company Details']
children = [widgets.Output() for tab in tabs]
tab = widgets.Tab()
tab.children = children
for i,title in enumerate(tabs):
    tab.set_title(i, title)

accordion_data = widgets.Accordion(children=[tab])
accordion_data.set_title(0, 'Data')

html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{filename}" download="{filename}">
<button class="p-Widget jupyter-widgets jupyter-button widget-button">Download Network</button>
</a>
</body>
</html>
'''

def generate_map():
    map_data,path_table = mapview.build_map(network, clear_widget=False) 
    hbox_map = widgets.HBox([path_table])
    vbox_map.children = [map_data, hbox_map]
    accordion_map.selected_index=0
    accordion_data.selected_index=0
    build_map_button.disabled = True
    build_map_button.button_style = "Success"
    with tab.children[0]:
        display(pd.DataFrame(network.company_ids))
    with tab.children[1]:
        display(pd.DataFrame(network.addresses))
    with tab.children[2]:
        display(pd.DataFrame(network.officer_ids))
    with tab.children[3]:
        display(pd.DataFrame(network.companies))
    file = str(company_text.value) + '.json'
    network.save(file, "./")
    html_button = html_buttons.format(filename=file)
    with download_link:
        display(HTML(html_button))

download_link = widgets.Output()
display(build_map_button, map_container)

In [ ]:
accordion_map

In [ ]:
accordion_data

In [ ]:
download_link